In [ ]:

#  At least 7 machines in the cluster and suggested machine type is n2-highmem-80.



In [ ]:

import os
import re
import pandas as pd

from katana import remote
from katana.remote import import_data

# Connect to the Katana Server
# client = remote.Client(address="host.docker.internal:8080")

my_client = remote.Client()

print(my_client)


In [ ]:
DB_NAME        = "my_db"
GRAPH_NAME1    = "my_graph1"
GRAPH_NAME2    = "my_graph2"
GRAPH_NAME3    = "my_graph3"

print("--")

In [ ]:
#  CONNECT TO GRAPH

# for l_graph in my_client.graphs():
#    if (l_graph.name == GRAPH_NAME1):
#       my_graph1=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
        
# for l_graph in my_client.graphs():
#    if (l_graph.name == GRAPH_NAME2):
#       my_graph2=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
        
for l_graph in my_client.graphs():
   if (l_graph.name == GRAPH_NAME3):
      my_graph3=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
        
# print(my_graph1)
# print(my_graph2)
print(my_graph3)


In [ ]:
# ##################################################################

In [ ]:

# graph_path = "gs://katana-internal5/Providence-Graphs/Hetio/"
# import_data.rdg(my_graph1, graph_path)

# graph_path = "gs://katana-internal5/Providence-Graphs/Hetio+GDSC/"
# import_data.rdg(my_graph2, graph_path)

graph_path = "gs://katana-internal48/graph/9JdxR9tyZaSocAbJLWzVAhjWL71YBQbVZNUi3wCJoxyF"
import_data.rdg(my_graph3, graph_path)

print("--")


In [ ]:

# display(print(my_graph1.num_nodes()))
# display(print(my_graph1.num_edges()))

# display(print(my_graph2.num_nodes()))
# display(print(my_graph2.num_edges()))

display(print(my_graph3.num_nodes()))
display(print(my_graph3.num_edges()))


In [ ]:

# my_graph1.schema(short_schema=True).view()

my_graph1.schema().view()


<img src="./01_Images/Result1.png" alt="Drawing" style="width: 1800px;"/>

In [ ]:

my_graph2.schema().view()


<img src="./01_Images/Result2.png" alt="Drawing" style="width: 1800px;"/>

In [ ]:

# my_graph3.schema().view()
my_graph3.schema(short_schema=True).view()


<img src="./01_Images/Result3.png" alt="Drawing" style="width: 1800px;"/>

In [ ]:
# ##################################################################

In [ ]:

%%time

l_query = """ 

   // optimization - much smaller set of essential genes so start here first before tcga
   // 
   MATCH (i:GDSC_IC50_RESULT) - [:HAS_CELL_LINE] -> (cl) <- [:HAS_CELL_LINE] - (ach:ACHILLES_ENTRY) - [:HAS_GENE] -> (essg:GENE)
   //
   // MATCH (ach:ACHILLES_ENTRY)-[:HAS_GENE]->(essg:GENE)
   
   WHERE
      i.tcga_cancer_name = 'SKCM'
   AND
      ach.Dependency > .8
   // AND
   //    essg.symbol = 'PDK1'
  
   // check for common essentials membership
   //
   AND
      essg.isCommonEssential='False'

   // this breaks it
   //
   // AND
   //    not ((ach)-[:HAS_ENTREZ_GENE]-(x:GENE))

   WITH DISTINCT essg, i.tcga_cancer_name AS cancerType, ach.Dependency AS depends
   LIMIT 1000
 
 
   //  /////////////////////////////////////////////////////////////
   
  
   MATCH (t1:TCGA_GENE) - [o:OBSERVED_MUTATION] -> (t2:TCGA_GENE)
   
   WHERE
      o.cancer_type = 'SKCM'
   AND
      (toFloat(o.Log2FoldChange )) > 0
   AND
      toFloat(o.MutSigPval) < .2
   AND
      t1.id = essg.symbol                                    // location SHOULD BE symbol
   AND o.cancer_type = cancerType
   // AND
   //    essg.symbol = 'RRM2'

   // this breaks it because some of genes are not in the tcga set
   //
   //
   // AND 
   //    t2.id  in  ['KRAS','MITF','CDKN2A','JAK2', 'PAX5', 'PPP6C', 'RAC1', 'SNX31', 'TACC1', 'STK19', 'ARID2']

   // confirmed these genes are in the tcga set - will comment in/out as needed
   // commented out right now cause dont find any hits at these threshold levels for just this small set of driver genes
   //
   AND
      t2.id = 'BRAF'                                          // in ['BRAF','KRAS','PPP6C','RAC1','STK19', 'ARID2']
  
   WITH DISTINCT essg, t2, o.Log2FoldChange AS fc, o.MutSigPval AS mutsigp, depends, cancerType


   //  /////////////////////////////////////////////////////////////
   
   
   MATCH (mutg:GENE)
   WHERE 
      t2.id = mutg.symbol
   WITH DISTINCT essg, mutg, cancerType, fc, mutsigp, depends
  
  
  // optional match (essg)-[r]-(mutg)
  //  with mutg,essg, cancerType,count(r) as r1c,fc,mutsigp,depends
   
 //  optional match (essg)-[r]-(x:GENE)-[r2]-(mutg)
 //   with mutg,essg,r1c,cancerType,count(r2) as r2c,fc,mutsigp,depends
   
 //  optional match (essg)-[r]-(x:GENE)-[r2]-(:GENE)-[r3]-(mutg)
 //   with mutg,essg,r1c,r2c,cancerType,count(r3) as r3c,fc,mutsigp,depends
  

   RETURN *  
   """


l_result=my_graph3.query(l_query, contextualize=True)

l_result.view()


<img src="./01_Images/Result4.png" alt="Drawing" style="width: 1800px;"/>